## Analisis NBA

In [1]:
import pandas as pd
import numpy as np

Leo las tablas para el analisis

In [2]:
all_seasons = pd.read_csv('all_seasons_limpiov2.csv')
game = pd.read_csv('game_limpio.csv')
line_score = pd.read_csv('line_score_filtrado.csv')
player = pd.read_csv('player_filtrado.csv')
team = pd.read_csv('team_clean.csv')

limito la tabla al periodo del analisis

In [3]:
all_seasons = all_seasons[all_seasons['season'] <= '2019-20']
all_seasons['season'].max()

'2019-20'

Creo una columna que agarre el año inicial de temporada

In [4]:
#funcion para extraer los primeros cuatro caracteres de 'season' 
def season_to_int(s):
    try:
        s = str(s)
        # buscar 4 dígitos al inicio
        return int(s[:4])
    except:
        return np.nan

all_seasons['season_year'] = all_seasons['season'].apply(season_to_int)

Asigno un puntaje a cada jugador por temporada

In [5]:
# Función que calcula un puntaje relativo a la temporada para cada jugador
def individual_score_season(columna):
    mean = columna.mean()
    std = columna.std(ddof=0)
    if pd.isna(std) or std == 0:
        # Si no hay variación, devolvemos 0 para todos
        return (columna - mean) * 0.0
    else:
        return (columna - mean) / std

# Calcular puntajes por temporada para cada atribuito
# Esto evita que cambios globales en la liga (p. ej. más puntos en los 2000s) alteren la comparación.
a_s_scores = all_seasons.copy()
Atributos = ['pts','reb','ast','net_rating','oreb_pct','dreb_pct','usg_pct','ts_pct','ast_pct']

for atrib in Atributos:
    a_s_scores[atrib + '_score'] = a_s_scores.groupby('season_year')[atrib].transform(individual_score_season)

# este for agarra cada atributo y genera una nueva columna en a_s_scores con el nombre del atributo + '_score'.
# en esas columnas se ve el puntaje relativo de cada jugador en relacion al promedio de todos los jugadores de la misma temporada

De esta manera logro poder comparar distintas unidades porque todo queda reflejado en desviacion estandar. De esta manera puedo trabajar con flotantes (como los puntos) y porcentajes (porcentaje de rebotos exitosos), con una unica medida.

Le asigno un unico puntaje a cada jugador, promediando sus atributos.

In [6]:
# estas son las columnas que reflejan el puntaje de cada jugador
score_columns = ['pts_score', 'reb_score', 'ast_score', 'net_rating_score', 'oreb_pct_score', 'dreb_pct_score', 'usg_pct_score', 'ts_pct_score', 'ast_pct_score']

a_s_scores['global_score'] = a_s_scores[score_columns].mean(axis=1)

Le colocamos, a cada jugador, la vara correspondiente a su equipo y temporada

In [7]:
# Usamos groupby + transform para que cada fila tenga su vara correspondiente
Vara = 0.10   # vara = percentil 10 (peor 10%)

a_s_scores['vara'] = a_s_scores.groupby(['team_abbreviation','season_year'])['global_score'].transform(lambda x: np.nanpercentile(x, Vara*100))
# aca estamos agrupando por equipo y temporarda, generando subgrupos de jugadores que comparten equipo y temporada.
# Tomo el 'global_score' de estos jugadores y devuelvo el percentil 10. Colocando este valor en una nueva columna 'vara' 

Califico en malo o bueno dependiendo si su puntaje es mejor o mayor a la vara

In [8]:
a_s_scores['rendimiento'] = np.where(a_s_scores['global_score'] < a_s_scores['vara'], 'Malo', 'Bueno')

In [9]:
a_s_scores['rendimiento'].isna().unique() #verifico que todos las filas tengan asignado un rendimiento

array([False])

Creo una tabla que muestre la vara correspondiente de cada equipo en cada temporada y cauntos jugadores estan por debajo (jugadores malos)

In [10]:
resumen = a_s_scores.groupby(['team_abbreviation','season_year']).agg( # agrupo por equipo y temporada, generando el subgrupo de jugadores que comparten
    n_players = ('player_name','count'), # agarra el subgrupo y cuenta cuantos nombres hay
    n_malos = ('rendimiento', lambda s: (s=='Malo').sum()), # agarra el subgrupo, toma el rendimientos y suma cuantos malos hay
    vara_value = ('vara','first')  # mismo valor para todo el grupo
).reset_index()

resumen

,team_abbreviation,season_year,n_players,n_malos,vara_value
0,ATL,1996,17,2,-0.498313
1,ATL,1997,16,2,-0.864000
2,ATL,1998,14,2,-0.455415
3,ATL,1999,13,2,-0.435460
4,ATL,2000,16,2,-0.699869
...,...,...,...,...,...
707,WAS,2015,17,2,-0.416098
708,WAS,2016,17,2,-0.425317
709,WAS,2017,15,2,-0.303885
710,WAS,2018,19,2,-0.767748


In [11]:
a_s_scores.to_csv('nba_puntaje_vara.csv', index=False)
resumen.to_csv('resumen.csv', index=False)
